In [184]:
# Import TensorFlow and NumPy
import tensorflow as tf
import numpy as np

# Set data type
DTYPE='float32'
tf.keras.backend.set_floatx(DTYPE) 

# Set constants
e0=1.
e1=10.
D=1.
k=0.55

# Define initial condition
def fun_u_0(x,u,u_x):
    return tf.stack([u_x[:,0]-k*np.sqrt(e1)*u[:,1],u_x[:,0]+k*np.sqrt(e0)*u_x[:,0]-2*k*np.sqrt(e0)],axis=1)
    #return np.array([u_x[0]-k*np.sqrt(e1)*u[1],u_x[0]+k*np.sqrt(e0)*u_x[0]-2*k*np.sqrt(e0)])
    #return u_x+1j*k*np.sqrt(e0)*u-2j*k*np.sqrt(e0)

# Define boundary condition
def fun_u_d(x,u,u_x):
    return tf.stack([u_x[:,0]+k*np.sqrt(e0)*u[:,1],u_x[:,1]+k*np.sqrt(e0)*u[:,0]-2*k*np.sqrt(e0)],axis=1)
    #return np.array([u_x[0]+k*np.sqrt(e0)*u[1],u_x[1]+k*np.sqrt(e0)*u[0]-2*k*np.sqrt(e0)])
    #return u_x-1j*k*np.sqrt(e0)*u

# Define residual of the PDE
def fun_r(x, u, u_x, u_xx):
    #return np.array([0.,0.])
    return tf.stack([u_xx[:,0]+k*k*e1*u[:,0],u_xx[:,1]+k*k*e1*u[:,1]],axis=1)
    #return np.array([u_xx[:,0]+k*k*e1*u[:,0],u_xx[:,1]+k*k*e1*u[:,1]])
    #return u_xx+k*k*e1*u


In [185]:
# Set number of data points
#N_0 = 50
N_r = 1000

# Set boundary
xmin = 0.
xmax = D

# Lower bounds
lb = tf.constant( xmin, dtype=DTYPE) 
# Upper bounds
ub = tf.constant( xmax, dtype=DTYPE)

# Set random seed for reproducible results
tf.random.set_seed(0)

# Draw uniform sample points for initial boundary data

#x_0 = tf.random.uniform((N_0,1), lb, ub, dtype=DTYPE)

# Evaluate intitial condition at x_0
#u_0 = fun_u_0()
#u_d = fun_u_d()


# Draw uniformly sampled collocation points
X_r = tf.random.uniform((N_r,1), lb, ub, dtype=DTYPE)
X_r = tf.concat([X_r, X_r], axis=1)


# Collect boundary and inital data in lists
X_data = [0, D]
#u_data = [u_0, u_d]
x_0=tf.zeros([1,2])
x_b=x_0=tf.reshape(tf.stack([ub,ub],axis=0),(1,2))

In [186]:
def init_model(num_hidden_layers=4, num_neurons_per_layer=20):
    # Initialize a feedforward neural network
    model = tf.keras.Sequential()

    # Input is two-dimensional (time + one spatial dimension)
    model.add(tf.keras.Input(2))

    # Introduce a scaling layer to map input to [lb, ub]
    scaling_layer = tf.keras.layers.Lambda(
                lambda x: (x - lb)/(ub - lb) ) 
    model.add(scaling_layer)

    # Append hidden layers
    for _ in range(num_hidden_layers):
        model.add(tf.keras.layers.Dense(num_neurons_per_layer,
            activation=tf.keras.activations.get('tanh'),
            kernel_initializer='glorot_normal'))

    # Output is one-dimensional
    model.add(tf.keras.layers.Dense(2))
    
    return model

In [187]:
def get_r(model,f):
    
    # A tf.GradientTape is used to compute derivatives in TensorFlow
    with tf.GradientTape(persistent=True) as tape:
        # Split t and x to compute partial derivatives
        if f==fun_r:
            x = X_r
        elif f==fun_u_0:
            x=x_0
        elif f==fun_u_d:
            x=x_b
        # Variables t and x are watched during tape
        # to compute derivatives u_t and u_x
        tape.watch(x)

        # Determine residual 
        u = model(x)
        # Compute gradient u_x within the GradientTape
        # since we need second derivatives
        u_x = tape.gradient(u, x)

    u_xx = tape.gradient(u_x, x)

    del tape
    if f == fun_r:
        return fun_r( x, u, u_x, u_xx)
    elif f == fun_u_0:
        print(fun_u_0(x,u,u_x))
        return fun_u_0(x,u,u_x)
    elif f ==fun_u_d:
        return fun_u_d(x,u,u_x)
    else:
        return 

In [188]:
'''def get_u_0(model,X_r):
    with tf.GradientTape(persistent=True) as tape:
        
        tape.watch(X_r)

        u = model(tf.stack( X_r, axis=1))

        # Compute gradient u_x within the GradientTape
        # since we need second derivatives
        u_x = tape.gradient(u, X_r)
    
    return '''

'def get_u_0(model,X_r):\n    with tf.GradientTape(persistent=True) as tape:\n        \n        tape.watch(X_r)\n\n        u = model(tf.stack( X_r, axis=1))\n\n        # Compute gradient u_x within the GradientTape\n        # since we need second derivatives\n        u_x = tape.gradient(u, X_r)\n    \n    return '

In [189]:
def compute_loss(model, X_r, X_data):
    
    # Compute phi^r
    #r = get_r(model, X_r,fun_r)
    #phi_r = tf.reduce_mean(tf.square(r))
    
    # Initialize loss
    loss = 0
    
    fun = [fun_u_0]
    # Add phi^0 and phi^b to the loss x_data 边界值
    for f in fun: 
        u_pred = get_r(model,f)
        print(u_pred)
        loss += tf.reduce_mean(tf.square(u_pred))


    return loss

In [190]:
def get_grad(model, X_r, X_data):
    
    with tf.GradientTape(persistent=True) as tape:
    # This tape is for derivatives with
    # respect to trainable variables
        tape.watch(model.trainable_variables)
        loss = compute_loss(model, X_r, X_data)

    g = tape.gradient(loss, model.trainable_variables)
    del tape

    return loss, g

In [191]:
# Initialize model aka u_\theta
model = init_model()

# We choose a piecewise decay of the learning rate, i.e., the
# step size in the gradient descent type algorithm
# the first 1000 steps use a learning rate of 0.01
# from 1000 - 3000: learning rate = 0.001
# from 3000 onwards: learning rate = 0.0005

lr = tf.keras.optimizers.schedules.PiecewiseConstantDecay([1000,3000],[1e-2,1e-3,5e-4])

# Choose the optimizer
optim = tf.keras.optimizers.Adam(learning_rate=lr)

In [192]:
from time import time

# Define one training step as a TensorFlow function to increase speed of training
@tf.function
def train_step():
    # Compute current loss and gradient w.r.t. parameters
    loss, grad_theta = get_grad(model, X_r, X_data)
    
    # Perform gradient descent step
    optim.apply_gradients(zip(grad_theta, model.trainable_variables))
    
    return loss

# Number of training epochs
N = 5000
hist = []

# Start timer
t0 = time()
with tf.GradientTape(persistent=True) as tape:
    # Split t and x to compute partial derivatives

    # Variables t and x are watched during tape
    # to compute derivatives u_t and u_x
    tape.watch(X_r)

    # Determine residual 
    u = model(X_r)
    # Compute gradient u_x within the GradientTape
    # since we need second derivatives
    u_x = tape.gradient(u, X_r)
    del tape

for i in range(N+1):
    
    loss = train_step()
    
    # Append current loss to hist
    hist.append(loss.numpy())
    
    # Output current loss after 50 iterates
    if i%50 == 0:
        print('It {:05d}: loss = {:10.8e}'.format(i,loss))
        
# Print computation time
print('\nComputation time: {} seconds'.format(time()-t0))

Tensor("stack:0", shape=(1, 2), dtype=float32)
Tensor("stack_1:0", shape=(1, 2), dtype=float32)
Tensor("stack:0", shape=(1, 2), dtype=float32)
Tensor("stack_1:0", shape=(1, 2), dtype=float32)
It 00000: loss = 6.20832622e-01
It 00050: loss = 4.82026808e-04
It 00100: loss = 5.72055660e-06
It 00150: loss = 2.80710086e-08
It 00200: loss = 9.13047415e-12
It 00250: loss = 3.48610030e-13
It 00300: loss = 1.67421632e-13
It 00350: loss = 5.68434189e-14
It 00400: loss = 8.88178420e-15
It 00450: loss = 7.54951657e-15
It 00500: loss = 4.44089210e-16
It 00550: loss = 7.28306304e-14
It 00600: loss = 3.01980663e-14
It 00650: loss = 2.66897615e-13
It 00700: loss = 2.31326069e-12
It 00750: loss = 7.54951657e-15
It 00800: loss = 8.74855743e-14
It 00850: loss = 2.22044605e-15
It 00900: loss = 1.15463195e-13
It 00950: loss = 6.29274410e-13
It 01000: loss = 2.66897615e-13
It 01050: loss = 2.22044605e-15
It 01100: loss = 8.88178420e-15
It 01150: loss = 2.70894418e-14
It 01200: loss = 2.22044605e-15
It 01250